In [67]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None) 
pd.set_option("display.max_colwidth", None)

In [68]:
boxscores_df = pd.read_csv("../tests/test_data/test_cleaned_boxscores.csv")
games_df = pd.read_csv("../tests/test_data/test_cleaned_games.csv")

boxscore_df.shape
# games_df.shape

(741569, 17)

In [69]:
# New DataFrame to get season start year from games
games_subset = games_df[["game_id", "season_start_year"]]
merged_df_to_get_year = boxscores_df.merge(games_subset, how="inner", on="game_id")

merged_df_to_get_year.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151265 entries, 0 to 151264
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   game_id                   151265 non-null  int64  
 1   team_name                 151265 non-null  object 
 2   player_name               151265 non-null  object 
 3   minutes_played            151265 non-null  object 
 4   field_goals               151265 non-null  int64  
 5   field_goals_attempted     151265 non-null  int64  
 6   three_pointers            151265 non-null  int64  
 7   three_pointers_attempted  151265 non-null  int64  
 8   free_throws               151265 non-null  int64  
 9   free_throws_attempted     151265 non-null  int64  
 10  total_rebounds            151265 non-null  int64  
 11  assists                   151265 non-null  int64  
 12  points                    151265 non-null  int64  
 13  is_starter                151265 non-null  i

In [78]:
# New DataFrame which contains in depth player stats
player_stats_df = (
    merged_df_to_get_year.groupby(["player_name", "season_start_year"])
        .agg({
            "points": "mean",
            "assists": "mean",
            "total_rebounds": "mean",
            "field_goals_percentage_%": "mean",
            "three_point_percentage_%": "mean",
            "free_throws_percentage_%": "mean",
            "three_pointers": "sum"
        })
        .round({
            "points": 1,
            "assists": 1,
            "total_rebounds": 1,
            "field_goals_percentage_%": 2,
            "free_throws_percentage_%": 2,
            "three_point_percentage_%": 2,
        })
        .reset_index()
        .rename(columns={
            "points": "points_per_game",
            "assists": "assists_per_game",
            "total_rebounds": "rebounds_per_game",
            "field_goals_percentage_%": "field_goal_%_per_game",
            "three_point_percentage_%": "three_point_%_per_game",
            "free_throws_percentage_%": "free_throws_%_per_game",
            "three_pointers": "total_three_pointers"
        })
)

player_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2585 entries, 0 to 2584
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player_name             2585 non-null   object 
 1   season_start_year       2585 non-null   int64  
 2   points_per_game         2585 non-null   float64
 3   assists_per_game        2585 non-null   float64
 4   rebounds_per_game       2585 non-null   float64
 5   field_goal_%_per_game   2585 non-null   float64
 6   three_point_%_per_game  2585 non-null   float64
 7   free_throws_%_per_game  2585 non-null   float64
 8   total_three_pointers    2585 non-null   int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 181.9+ KB


In [71]:
# New DataFrame to get the team name from boxscores
boxscores_subset = boxscores_df[["game_id", "team_name"]]
merged_df_to_get_team_name = games_df.merge(boxscores_subset, how="inner", on="game_id")

merged_df_to_get_team_name.head()

,season_start_year,away_team,points_away,home_team,points_home,date_time,game_id,team_name
0,2015,Cleveland Cavaliers,95,Chicago Bulls,97,27-10-2015,23859,Cleveland Cavaliers
1,2015,Cleveland Cavaliers,95,Chicago Bulls,97,27-10-2015,23859,Cleveland Cavaliers
2,2015,Cleveland Cavaliers,95,Chicago Bulls,97,27-10-2015,23859,Cleveland Cavaliers
3,2015,Cleveland Cavaliers,95,Chicago Bulls,97,27-10-2015,23859,Cleveland Cavaliers
4,2015,Cleveland Cavaliers,95,Chicago Bulls,97,27-10-2015,23859,Cleveland Cavaliers


In [75]:
# Function to determine winner of a game
def determine_winner(row):
    if row["team_name"] == row["home_team"]:
        return int(row["points_home"] > row["points_away"])
    elif row["team_name"] == row["away_team"]:
        return int(row["points_away"] > row["points_home"])
    else:
        return None

# New column
merged_df_to_get_team_name["won_game"] = merged_df_to_get_team_name.apply(determine_winner, axis=1)

# Drop duplicates
team_wins_df = merged_df_to_get_team_name.drop_duplicates()

# Count total number of wins a team has
total_wins_df = (
    team_wins_df.groupby(["season_start_year", "team_name"])["won_game"]
        .sum()
        .reset_index()
        .rename(columns={
            "won_game": "total_wins"
        })
)

# Count total number of games a team played per year
games_played_df = (
    team_wins_df.groupby(["season_start_year", "team_name"])["won_game"]
        .count()
        .reset_index()
        .rename(columns={
            "won_game": "total_games"
        })
)

# Create a new DataFrame to have the team stats for each year
team_stats_df = pd.merge(games_played_df, total_wins_df, on=["season_start_year", "team_name"])
# Add a new column to find out how many losses a team had
team_stats_df["total_losses"] = team_stats_df["total_games"] - team_stats_df["total_wins"]
# Add a new column to calculate the win percentage
team_stats_df["win_%"] = (team_stats_df["total_wins"] / team_stats_df["total_games"] * 100).round(2)


team_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season_start_year  150 non-null    int64  
 1   team_name          150 non-null    object 
 2   total_games        150 non-null    int64  
 3   total_wins         150 non-null    int64  
 4   total_losses       150 non-null    int64  
 5   win_%              150 non-null    float64
dtypes: float64(1), int64(4), object(1)
memory usage: 7.2+ KB


In [73]:
# Save player stats DataFrame to a CSV
FILE_PATH = "../tests/test_data/test_player_stats.csv"
player_stats_df.to_csv(FILE_PATH, index=False)

In [74]:
# Save player stats DataFrame to a CSV
FILE_PATH = "../tests/test_data/test_team_stats.csv"
team_stats_df.to_csv(FILE_PATH, index=False)